# Test of GraphSAGE
- use DGL
- predict `graphs`
- valid, test data are in the training dataset

In [15]:
import os
import dgl
import json
import torch
import torch as th
import dgl.nn as dglnn
# from tqdm import tqdm
from tqdm.notebook import tqdm  # 使用 notebook 版本的 tqdm
import torch.nn as nn
from dgl.nn import GraphConv, GATConv, SAGEConv
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup

- check the GPU and assign the GPU by the best memory usage

In [16]:
import subprocess
import torch

def get_free_gpu():
    try:
        # Run nvidia-smi command to get GPU details
        _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
        command = "nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader"
        memory_free_info = _output_to_list(subprocess.check_output(command.split())) 
        memory_free_values = [int(x) for i, x in enumerate(memory_free_info)]
        
        # Get the GPU with the maximum free memory
        best_gpu_id = memory_free_values.index(max(memory_free_values))
        return best_gpu_id
    except:
        # If any exception occurs, default to GPU 0 (this handles cases where nvidia-smi isn't installed)
        return 0

if torch.cuda.is_available():
    # Get the best GPU ID based on free memory and set it
    best_gpu_id = get_free_gpu()
    device = torch.device(f"cuda:{best_gpu_id}")
else:
    device = torch.device("cpu")
    print("there's no available GPU")

# device = torch.device(f"cuda:{1}")
print(device)


cuda:1


## Fix the seed

In [3]:
import numpy as np
import torch
import random

#fix seed
def same_seeds(seed = 8787):
    torch.manual_seed(seed)
    # random.seed(seed) 
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Data Loader

In [4]:
class GraphDataset(Dataset):
    def __init__(self, data_list, device):
        self.data_list = data_list
        self.device = device

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        data = self.data_list[idx]

        g = dgl.graph((th.tensor(data["edge_index"][0]), th.tensor(data["edge_index"][1])), num_nodes=data["num_nodes"]).to(self.device)

        g.ndata['feat'] = th.tensor(data["node_feat"]).to(self.device)
        g.edata['feat'] = th.tensor(data["edge_attr"]).to(self.device)  # Add edge features to graph

        return g, th.tensor(data["label"]).to(self.device)


def collate(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.tensor(labels)


In [5]:
datasets = ['train', 'valid', 'test']
dataset_data = {}

for dataset_name in tqdm(datasets):
    file_path = f"../../data_processing/dgl/data_new/exp1-2/training_data/exp_2/transR_50/{dataset_name}.jsonl"
    
    print(file_path)
    with open(file_path) as f:
        data_list = [json.loads(line) for line in tqdm(f, position=0, leave=True)]
    
    dataset_data[dataset_name] = GraphDataset(data_list, device)

print("Datasets loaded!")

  0%|          | 0/3 [00:00<?, ?it/s]

../../data_processing/dgl/data_new/exp1-2/training_data/exp_2/transR_50/train.jsonl


0it [00:00, ?it/s]

../../data_processing/dgl/data_new/exp1-2/training_data/exp_2/transR_50/valid.jsonl


0it [00:00, ?it/s]

../../data_processing/dgl/data_new/exp1-2/training_data/exp_2/transR_50/test.jsonl


0it [00:00, ?it/s]

Datasets loaded!


- choose batch size

In [6]:
def create_dataloaders(batch_size, shuffle=True):
    dataloaders = {}
    for dataset_name, dataset in dataset_data.items():
        # do not shuffle the testing dataset
        if dataset_name == "test":
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)    
        else:
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate)
    return dataloaders

dataloaders = create_dataloaders(8)

- Turn the print message to a log file

In [7]:
import datetime

now = datetime.datetime.now()

formatted_time = now.strftime("%m%d_%H:%M")

log_file_path = f"../log_message/{formatted_time}_GraphSAGE_transR_50.log"

def add_log_msg(msg, log_file_path=log_file_path):
    with open(log_file_path, 'a') as f:
        f.write(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}\n')
    print(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}')

print(log_file_path)

../log_message/0912_22:49_GraphSAGE_transR_50.log


### Model

In [8]:
class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(GraphSAGE, self).__init__()
#         self.layer1 = dglnn.SAGEConv(in_dim, hidden_dim, 'mean')
#         self.layer2 = dglnn.SAGEConv(hidden_dim, out_dim, 'mean')  # Output layer
        self.layer1 = dglnn.SAGEConv(in_dim, hidden_dim, 'pool')
        self.layer2 = dglnn.SAGEConv(hidden_dim, out_dim, 'pool')  # Output layer

    def forward(self, g, inputs):
        h = self.layer1(g, inputs)
        h = torch.relu(h)
        h = self.layer2(g, h)  # You can apply another non-linearity here if needed
        
        g.ndata['h'] = h
        hg = dgl.mean_nodes(g, 'h')
        return hg


- Model Forward  

In [9]:
def model_fn(data, model, criterion, device, count=1, which_type='train'):
    """Forward a batch through the model."""
    batched_g, labels = data
    batched_g = batched_g.to(device)
    
    labels = labels.to(device)
#     labels = labels.float()

    logits = model(batched_g, batched_g.ndata['feat'].float()) # for GAT
#     logits = logits.mean(dim=1)
#     print(labels)
#     print(labels.shape)
#     print(logits)
#     print(logits.shape)

    
    loss = criterion(logits, labels)

    # Get the class id with the highest probability
    preds = logits.argmax(1) # for GAT
#     print(preds)
#     preds = logits.argmax() # for graphSAGE
    
    '''
    问题出在GAT和GraphSAGE模型的输出形状上。
    在你的 GAT 模型中，最后一层的输出是一个一维的张量，因此 logits 是一个形状为 (batch_size,) 的张量。这是因为 GAT 模型最后一层的输出被处理成了一个节点的预测，而不是整个图的预测。
    在 GraphSAGE 模型中，最后一层的输出形状是 (num_nodes, out_dim)，也就是说，它返回了整个图的节点级别的预测。因此，logits 是一个二维的张量，形状为 (num_nodes, out_dim)。
    当你尝试在一个一维张量上调用 logits.argmax(1) 时，会产生维度错误，因为它期望一个二维张量来执行操作。
    所以，在 GraphSAGE 模型中，你应该将以下行：
    '''
    
    
    # Compute accuracy
    accuracy = torch.mean((preds == labels).float())
    
    if which_type == 'validation' and count % 1000 == 0:
        add_log_msg(f"labels of Validation: {labels} {labels.shape}")
        add_log_msg(f"predicted of Validation: {preds} {preds.shape}")
        
    elif which_type == 'test'  and count % 1000 == 0:
        add_log_msg(f"labels of Test: {labels} {labels.shape}")
        add_log_msg(f"predicted of Test: {preds} {preds.shape}")
        
    if count % 5000 == 0: 
        add_log_msg(f"labels of {count}: {labels} {labels.shape}")
        add_log_msg(f"predicted of {count}: {preds} {preds.shape}")
        
    return loss, accuracy, preds

### Training

- Fix the seed and save the model.state_dict that contains the initial weight

In [10]:
seed = 8787
same_seeds(seed)

model = GraphSAGE(in_dim=50, hidden_dim=128, out_dim=167)
torch.save(model.state_dict(), 'model3_initial(graphsage)/initial_weight.pth')

In [11]:
model.layer1.fc_self.weight

Parameter containing:
tensor([[-0.1415, -0.1232, -0.2210,  ..., -0.1132,  0.1040,  0.1068],
        [ 0.2094, -0.0312,  0.1230,  ..., -0.0153, -0.0057,  0.1573],
        [-0.1400,  0.2284, -0.0143,  ..., -0.1863, -0.2516, -0.1591],
        ...,
        [ 0.1911,  0.0178, -0.2430,  ..., -0.1795, -0.0926, -0.0837],
        [ 0.1862,  0.0516,  0.0851,  ...,  0.2165,  0.2184, -0.2323],
        [ 0.0705, -0.0225,  0.0456,  ..., -0.0348, -0.0728,  0.2025]],
       requires_grad=True)

- Check if model really load the model_dict

In [12]:
model = GraphSAGE(in_dim=50, hidden_dim=128, out_dim=167)
model.load_state_dict(torch.load('model3_initial(graphsage)/initial_weight.pth'))
model.layer1.fc_self.weight

Parameter containing:
tensor([[-0.1415, -0.1232, -0.2210,  ..., -0.1132,  0.1040,  0.1068],
        [ 0.2094, -0.0312,  0.1230,  ..., -0.0153, -0.0057,  0.1573],
        [-0.1400,  0.2284, -0.0143,  ..., -0.1863, -0.2516, -0.1591],
        ...,
        [ 0.1911,  0.0178, -0.2430,  ..., -0.1795, -0.0926, -0.0837],
        [ 0.1862,  0.0516,  0.0851,  ...,  0.2165,  0.2184, -0.2323],
        [ 0.0705, -0.0225,  0.0456,  ..., -0.0348, -0.0728,  0.2025]],
       requires_grad=True)

### training part
- hidden_dim = 128
- batch_size = 8

In [ ]:
import csv
import pandas as pd
from sklearn.metrics import classification_report
from torch.optim import AdamW, lr_scheduler

seed = 8787
same_seeds(seed)

model = GraphSAGE(in_dim=50, hidden_dim=128, out_dim=167)
# in_dim means the dimension of the node_feat(50 dim, since the 50-dim embedding)
# out_dim means the # of the categories -> 168 for out tasks
model.load_state_dict(torch.load('model3_initial(graphsage)/initial_weight.pth'))
best_model_path = "../checkpoint_graphSAGE/best_model_GraphSAGE_transR_50-1.pt"

model = model.to(device)

# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
optimizer = AdamW(model.parameters(), lr=1e-4)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=18, num_training_steps=total_steps)

# T_max control the period of the lr changing -> set 1/10 first
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0, last_epoch=- 1, verbose=False)


criterion = nn.CrossEntropyLoss()
# criterion = torch.nn.BCEWithLogitsLoss()

total_steps = 100

# save the best model
best_val_loss = float('inf')
patience = 10  # Number of epochs with no improvement after which training will be stopped.
waiting = 0  # The number of epochs with no improvement so far.


# Training Part
for epoch in tqdm(range(total_steps)):
    # Train
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
    
    for data in tqdm(dataloaders['train'], desc="Training", position=0, leave=True):
        num_batches += 1
        loss, accuracy, _ = model_fn(data, model, criterion, device, num_batches, which_type='train')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()

        
    scheduler.step()
    add_log_msg(f"total batches: {num_batches}")

    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches

    add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')

    
    # Validation Part
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    num_batches = 0


    with torch.no_grad():
        for data in tqdm(dataloaders['valid'], desc="Validation", position=0, leave=True):
            loss, accuracy, _ = model_fn(data, model, criterion, device, num_batches, which_type='validation')
            total_accuracy += accuracy.item()
            total_loss += loss.item()
            num_batches += 1

    avg_accuracy = total_accuracy / num_batches
    current_loss = total_loss / num_batches
    
    add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}\n')
    
            
    if current_loss < best_val_loss:
        best_val_loss = current_loss
        waiting = 0
        
        if os.path.exists(best_model_path):
            os.remove(best_model_path)
            add_log_msg("Find a better model!!")

        torch.save(model.state_dict(), best_model_path)

#         print(best_model_path)

    else:
        waiting += 1
        if waiting >= patience:
            add_log_msg("============================== Early stopping ==================================")
            break

  0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/16600 [00:00<?, ?it/s]

09/12/2023, 22:50:20# labels of 5000: tensor([ 34, 100, 143,  58,   7,  33,  17,  50], device='cuda:2') torch.Size([8])
09/12/2023, 22:50:20# predicted of 5000: tensor([ 91,  91,  91,  91, 134,  91, 134, 106], device='cuda:2') torch.Size([8])
09/12/2023, 22:51:35# labels of 10000: tensor([113,  19, 104, 154,  11,  57,   4,  78], device='cuda:2') torch.Size([8])
09/12/2023, 22:51:35# predicted of 10000: tensor([131,  19,  36, 154,  49,  36,  36, 127], device='cuda:2') torch.Size([8])
09/12/2023, 22:52:52# labels of 15000: tensor([118, 145,  50, 112, 143,   2, 117, 110], device='cuda:2') torch.Size([8])
09/12/2023, 22:52:52# predicted of 15000: tensor([118,  80, 117,  83,  83,  83, 117, 108], device='cuda:2') torch.Size([8])
09/12/2023, 22:53:18# total batches: 16600
09/12/2023, 22:53:18# Epoch 0 | Train Loss: 3.7291 | Train Accuracy: 0.3031


Validation:   0%|          | 0/2075 [00:00<?, ?it/s]

09/12/2023, 22:53:18# labels of Validation: tensor([114, 151,  84,  64, 131,  55, 119,  98], device='cuda:2') torch.Size([8])
09/12/2023, 22:53:18# predicted of Validation: tensor([108,  57,  84, 108, 108,  57,  57,  98], device='cuda:2') torch.Size([8])
09/12/2023, 22:53:18# labels of 0: tensor([114, 151,  84,  64, 131,  55, 119,  98], device='cuda:2') torch.Size([8])
09/12/2023, 22:53:18# predicted of 0: tensor([108,  57,  84, 108, 108,  57,  57,  98], device='cuda:2') torch.Size([8])
09/12/2023, 22:53:29# labels of Validation: tensor([ 77,  63,  71, 131, 163, 152,  26, 145], device='cuda:2') torch.Size([8])
09/12/2023, 22:53:29# predicted of Validation: tensor([ 77,  63,  71, 108,  57,  57,  26, 145], device='cuda:2') torch.Size([8])
09/12/2023, 22:53:41# labels of Validation: tensor([ 56,  66, 134,  74,  81, 151, 163,  44], device='cuda:2') torch.Size([8])
09/12/2023, 22:53:41# predicted of Validation: tensor([ 56,  66, 134,  57,  57,  57,  57,  57], device='cuda:2') torch.Size([8]

...

09/13/2023, 02:20:49# labels of 5000: tensor([ 11, 110,  38, 143, 145, 115,   0,  79], device='cuda:2') torch.Size([8])
09/13/2023, 02:20:49# predicted of 5000: tensor([152, 110, 152, 152, 145, 115,   0,  79], device='cuda:2') torch.Size([8])
09/13/2023, 02:22:06# labels of 10000: tensor([ 97,  53,  29,  32, 156,  53,  29,  63], device='cuda:2') torch.Size([8])
09/13/2023, 02:22:06# predicted of 10000: tensor([ 30,  48,  29,  32, 156, 112,  29,  63], device='cuda:2') torch.Size([8])
09/13/2023, 02:23:25# labels of 15000: tensor([ 42, 100, 148,  74,  77,   6, 105,  57], device='cuda:2') torch.Size([8])
09/13/2023, 02:23:25# predicted of 15000: tensor([144, 100, 148,  49,  77,   6, 105,  38], device='cuda:2') torch.Size([8])
09/13/2023, 02:23:48# total batches: 16600
09/13/2023, 02:23:48# Epoch 46 | Train Loss: 1.4449 | Train Accuracy: 0.6223


Validation:   0%|          | 0/2075 [00:00<?, ?it/s]

09/13/2023, 02:23:48# labels of Validation: tensor([113,  76,  33,  51,  69,  15,  81,  62], device='cuda:2') torch.Size([8])
09/13/2023, 02:23:48# predicted of Validation: tensor([ 64, 119, 121,  51,  69,  15, 158,  62], device='cuda:2') torch.Size([8])
09/13/2023, 02:23:48# labels of 0: tensor([113,  76,  33,  51,  69,  15,  81,  62], device='cuda:2') torch.Size([8])
09/13/2023, 02:23:48# predicted of 0: tensor([ 64, 119, 121,  51,  69,  15, 158,  62], device='cuda:2') torch.Size([8])
09/13/2023, 02:23:59# labels of Validation: tensor([ 20,  67, 133,  90,  16, 117, 142, 160], device='cuda:2') torch.Size([8])
09/13/2023, 02:23:59# predicted of Validation: tensor([ 20,  67, 133,  64,  16, 117, 119, 160], device='cuda:2') torch.Size([8])
09/13/2023, 02:24:11# labels of Validation: tensor([160, 110, 140, 142, 147, 129,  56,   1], device='cuda:2') torch.Size([8])
09/13/2023, 02:24:11# predicted of Validation: tensor([160,  64, 140, 121, 147, 129,  56, 158], device='cuda:2') torch.Size([8]

Training:   0%|          | 0/16600 [00:00<?, ?it/s]

09/13/2023, 02:25:24# labels of 5000: tensor([ 70, 123,  95,  27, 161,   3,  57,  50], device='cuda:2') torch.Size([8])
09/13/2023, 02:25:24# predicted of 5000: tensor([ 70, 123,  95,  27, 161,   3,  87,  50], device='cuda:2') torch.Size([8])


### test of valid and test part is ``graph``

- 60 APs in training x 10000times
- 5 APs in validation x 4 times
- 3 APs in test x 4 times
- Batch size = 4

In [17]:
from sklearn.metrics import classification_report

model = GraphSAGE(in_dim=50, hidden_dim=128, out_dim=167)
criterion = nn.CrossEntropyLoss()


# load the pretrained model
pretrained_model_path = '../checkpoint_graphSAGE/best_model_GraphSAGE_transR_50-1.pt'
model.load_state_dict(torch.load(pretrained_model_path))

model.to(device)
model.eval()

total = 0
correct = 0
count = 0

true_labels = []
predicted_labels = []

with torch.no_grad():
    for data in tqdm(dataloaders['test'], desc="Testing", position=0, leave=True):
#         print(f"data:{data[1]}")
        loss, accuracy, predicted = model_fn(data, model, criterion, device, count, which_type='test')
        labels = data[1].to(device)
        
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())
        
        if count % 5000 == 0:
            add_log_msg(f"labels: {labels} {labels.shape}")
            add_log_msg(f"predicted: {predicted} {predicted.shape}")
            
        count += 1
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

add_log_msg(f'Test Accuracy: {100 * correct / total} %\n\n\n')


# ======================================== handlig the output excel files ========================================
mapping_file = './new_mapping.txt'
label_mapping = {}
with open(mapping_file, 'r') as f:
    for line in f:
        parts = line.strip().split(': ')
        label_mapping[int(parts[1])] = parts[0]
        
# 将映射后的标签应用到true和predicted标签列表
mapped_true_labels = [label_mapping[label] for label in true_labels]
mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

# 生成Scikit-learn报告信息的DataFrame
report_data = classification_report(mapped_true_labels, mapped_predicted_labels, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()

# mapped_true_labels_np = np.array(mapped_true_labels)
# mapped_predicted_labels_np = np.array(mapped_predicted_labels)

# print("mapped_true_labels 的形状:", mapped_true_labels_np.shape)
# print("mapped_predicted_labels 的形状:", mapped_predicted_labels_np.shape)

report_folder = 'classification_report'
os.makedirs(report_folder, exist_ok=True)

count = 0
while True:
    report_filename = f'classification_report-transR_50-graphSAGE-{count}.xlsx'
    labels_filename = f'mapped_true_predicted_labels-transR_50-graphSAGE-{count}.xlsx'
    
    report_path = os.path.join(report_folder, report_filename)
    labels_path = os.path.join(report_folder, labels_filename)
    
    if not os.path.exists(report_path) and not os.path.exists(labels_path):
        break
    count += 1

    
report_df.to_excel(report_path, index_label='Label')

mapped_labels_df = pd.DataFrame({'true_label': mapped_true_labels, 'predicted_label': mapped_predicted_labels})
mapped_labels_df.to_excel(labels_path, index=False)

add_log_msg(f"report path: {report_path}")
add_log_msg(f"label path: {labels_path}")

mapped_report = classification_report(mapped_true_labels, mapped_predicted_labels)
add_log_msg(f"mapped_report:\n{mapped_report}")

Testing:   0%|          | 0/2075 [00:00<?, ?it/s]

09/13/2023, 14:01:03# labels of Test: tensor([65, 65, 65, 65, 65, 65, 65, 65], device='cuda:1') torch.Size([8])
09/13/2023, 14:01:03# predicted of Test: tensor([65, 65, 65, 65, 65, 65, 65, 65], device='cuda:1') torch.Size([8])
09/13/2023, 14:01:03# labels of 0: tensor([65, 65, 65, 65, 65, 65, 65, 65], device='cuda:1') torch.Size([8])
09/13/2023, 14:01:03# predicted of 0: tensor([65, 65, 65, 65, 65, 65, 65, 65], device='cuda:1') torch.Size([8])
09/13/2023, 14:01:03# labels: tensor([65, 65, 65, 65, 65, 65, 65, 65], device='cuda:1') torch.Size([8])
09/13/2023, 14:01:03# predicted: tensor([65, 65, 65, 65, 65, 65, 65, 65], device='cuda:1') torch.Size([8])
09/13/2023, 14:01:11# labels of Test: tensor([82, 82, 82, 82, 82, 82, 82, 82], device='cuda:1') torch.Size([8])
09/13/2023, 14:01:11# predicted of Test: tensor([82, 82, 82, 82, 82, 82, 82, 82], device='cuda:1') torch.Size([8])
09/13/2023, 14:01:20# labels of Test: tensor([162, 162, 162, 162, 162, 162, 162, 162], device='cuda:1') torch.Size

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


09/13/2023, 14:01:26# report path: classification_report/classification_report-transR_50-graphSAGE-3.xlsx
09/13/2023, 14:01:26# label path: classification_report/mapped_true_predicted_labels-transR_50-graphSAGE-3.xlsx


/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


09/13/2023, 14:01:26# mapped_report:
                                                precision    recall  f1-score   support

T1003.001_0ef4cc7b-611c-4237-b20b-db36b6906554       1.00      1.00      1.00       100
    T1003.001_35d92515122effdd73801c6ac3021da7       1.00      1.00      1.00       100
    T1003.002_5a484b65c247675e3b7ada4ba648d376       1.00      1.00      1.00       100
    T1003.002_7fa4ea18694f2552547b65e23952cabb       1.00      1.00      1.00       100
    T1003.003_9f73269695e54311dd61dc68940fb3e1       0.00      0.00      0.00       100
    T1003.003_f049b89533298c2d6cd37a940248b219       0.01      0.02      0.02       100
        T1003_18f31c311ac208802e88ab8d5af8603e       1.00      1.00      1.00       100
        T1007_9d03c91bdae5a80f17f89c987942b5a8       1.00      1.00      1.00       100
    T1007_c6607391-d02c-44b5-9b13-d3492ca58599       0.01      0.02      0.02       100
        T1007_d6bb2a19da7246731ed9c44831b135f8       0.45      0.27      0.34     

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
